In [13]:
import pandas as pd
from pandas import Series, DataFrame
import numpy as np
import pprint
from datetime import datetime, timedelta
#import tim
from dfply import *
import matplotlib.pyplot as plt
import seaborn as sns
import impyute as impy
from impyute.imputation.cs import mice
#from sqlalchemy import create_engine
#https://chmodi.tistory.com/114

In [14]:
import psycopg2
from sshtunnel import SSHTunnelForwarder

try:
    tunnel= SSHTunnelForwarder(
         ('210.91.223.248', 35430),
         #ssh_private_key="</path/to/private/ssh/key>",
         ### in my case, I used a password instead of a private key
         ssh_username="medicalai",
         ssh_password="medicalai731!", 
         remote_bind_address=('localhost', 35432))
         
    tunnel.start()
    print("tunnel connected")

    params = {
             'database': 'mimiciv',
             'user': 'mai_admin',
             'password': 'admin1q2w!@',
             'host': 'localhost',
             'port': tunnel.local_bind_port
             }

    conn = psycopg2.connect(**params)
    conn.autocommit = True
    curs = conn.cursor()
    print("database connected")

except Exception as e:
    print("Connection Failed due to {e}")

# Remember to close the SSH tunnel
#tunnel.close()


tunnel connected
database connected


In [15]:
curs.execute("select * from mimiciv_derived.gcs;")
columns_name = [desc[0] for desc in curs.description]
gcs = pd.DataFrame(curs.fetchall(), columns=columns_name) 

In [16]:
gcs.to_csv('../outputs/gcs.csv', index=False, encoding='utf-8-sig')
gcs = pd.read_csv('../outputs/gcs.csv', parse_dates=['charttime'])

In [17]:
gcs

,subject_id,stay_id,charttime,gcs,gcs_motor,gcs_verbal,gcs_eyes,gcs_unable
0,12466550,30000153,2174-09-29 12:45:00,15.0,5.0,0.0,3.0,1
1,12466550,30000153,2174-09-29 16:26:00,15.0,6.0,0.0,4.0,1
2,12466550,30000153,2174-09-29 17:37:00,15.0,6.0,0.0,4.0,1
3,12466550,30000153,2174-09-29 18:00:00,9.0,5.0,1.0,3.0,0
4,12466550,30000153,2174-09-29 19:00:00,9.0,5.0,1.0,3.0,0
...,...,...,...,...,...,...,...,...
1637758,17840864,39999810,2115-12-05 00:00:00,14.0,6.0,5.0,3.0,0
1637759,17840864,39999810,2115-12-05 04:00:00,14.0,6.0,5.0,3.0,0
1637760,17840864,39999810,2115-12-05 08:28:00,15.0,6.0,5.0,4.0,0
1637761,17840864,39999810,2115-12-05 12:00:00,15.0,6.0,5.0,4.0,0


In [18]:
subjectlist1 = pd.read_csv('./data/subjectlist1.csv', parse_dates=['admittime','intubationtime','dischtime', 'deathtime','extubationtime','reintubation_eventtime'])
sub1list = subjectlist1 >> select('subject_id','hadm_id','stay_id')

In [19]:
sub1_gcs = sub1list >> left_join(gcs, by=['subject_id','stay_id'])

In [21]:
sub1_gcs.to_csv('../outputs/sub1_gcs.csv', index=False, encoding='utf-8-sig')

In [22]:
sub1_gcs.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 572393 entries, 0 to 572392
Data columns (total 9 columns):
 #   Column      Non-Null Count   Dtype         
---  ------      --------------   -----         
 0   subject_id  572393 non-null  int64         
 1   hadm_id     572393 non-null  int64         
 2   stay_id     572393 non-null  int64         
 3   charttime   572381 non-null  datetime64[ns]
 4   gcs         572381 non-null  float64       
 5   gcs_motor   570538 non-null  float64       
 6   gcs_verbal  571134 non-null  float64       
 7   gcs_eyes    571745 non-null  float64       
 8   gcs_unable  572381 non-null  float64       
dtypes: datetime64[ns](1), float64(5), int64(3)
memory usage: 39.3 MB
